In [2]:
import csv

'Load the raw data'
def load_samples():
    samples = []
    with open('./data/driving_log.csv') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)
        for sample in reader:
            samples.append(sample)
    return samples

samples = load_samples()
print(len(samples))


8036


In [3]:
%%time
import cv2
import matplotlib.pyplot as plt

'Function to extract images and steering angles from raw data'
def load_images(samples):
    images = []
    steering_angles = []
    num_samples = len(samples)
    for i, sample in enumerate(samples):
        print('Reading Sample: '+str(i)+' of '+str(num_samples), end='\r')
        for j in range(3):
            source_path = sample[j]
            filename = source_path.split('/')[-1]
            image_path = './data/IMG/' + filename
            image = cv2.imread(image_path)
            images.append((cv2.cvtColor(image, cv2.COLOR_BGR2RGB)))

        correction = 0.2
        steering_angle_center = float(sample[3])
        steering_angle_left = steering_angle_center + correction
        steering_angle_right = steering_angle_center - correction
        steering_angles.append(steering_angle_center)
        steering_angles.append(steering_angle_left)
        steering_angles.append(steering_angle_right)    
    return images, steering_angles

images, steering_angles = load_images(samples)
print(len(images), len(steering_angles))


24108 241085 of 8036
Wall time: 1min 56s


In [4]:
%%time
import numpy as np

'Function to augment the images and steering angles'
def augment_data(images, steering_angles):
    augmented_images = []
    augmented_steering_angles = []
    for image, steering_angle in zip(images, steering_angles):
        augmented_images.append(image)
        augmented_steering_angles.append(steering_angle)
        flipped_image = cv2.flip(image, 1)
        flipped_steering_angle = steering_angle * -1.0
        augmented_images.append(flipped_image)
        augmented_steering_angles.append(flipped_steering_angle)
    return augmented_images, augmented_steering_angles

augmented_images, augmented_steering_angles = augment_data(images, steering_angles)
print(len(augmented_images), len(augmented_steering_angles))
x_train = np.array(augmented_images)
y_train = np.array(augmented_steering_angles)
print (x_train.shape)
print (y_train.shape)


48216 48216
(48216, 160, 320, 3)
(48216,)
Wall time: 10.8 s


In [ ]:
import sklearn
from sklearn.model_selection import train_test_split

'Generator function to process data in batches'
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while True:                                                # Loop forever so the generator never terminates
        sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            print(offset, num_samples, batch_size)
            images = []
            angles = []
            for batch_sample in batch_samples:
                images, steering_angles = load_images(batch_sample)
                augmented_images, augmented_steering_angles = augment_data(images, steering_angles)

            X_train = np.array(augmented_images)
            y_train = np.array(augmented_steering_angles)
            yield sklearn.utils.shuffle(x_train, y_train)

In [5]:
import matplotlib.pyplot as plt
import os

'Plot the training and validation loss for each epoch'
def plot_history(history_object, save_path, title):
    plt.plot(history_object.history['loss'])
    plt.plot(history_object.history['val_loss'])
    plt.title('model mean squared error loss')
    plt.ylabel('mean squared error loss')
    plt.xlabel('epoch')
    plt.legend(['training set', 'validation set'], loc='upper right')
    plt.savefig(os.path.join(save_path + title + '.png'))
    plt.show()

In [6]:
import keras
from keras.models import Sequential, Model
from keras.layers import Lambda, Flatten, Dense, Activation, Dropout
from keras.layers.convolutional import Convolution2D, Cropping2D
from keras.layers.pooling import MaxPooling2D

'Simple Regression Model'
def simple_model():
    model = Sequential()
    model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160, 320, 3)))
    model.add(Cropping2D(cropping=((70, 25), (0, 0))))
    model.add(Flatten())
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

'LeNet Model'
def lenet_model():
    model = Sequential()
    model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160, 320, 3)))
    model.add(Cropping2D(cropping=((70, 25), (0, 0))))
    model.add(Convolution2D(filters=6, kernel_size=(5, 5), strides=(2, 2), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Convolution2D(16, (5, 5), strides=(2, 2), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

'NVIDIA End-to-End Model'
def nvidia_model():
    model = Sequential()
    model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160, 320, 3)))
    model.add(Cropping2D(cropping=((50, 20), (0, 0))))
    model.add(Convolution2D(filters=24, kernel_size=(5, 5), strides=(2, 2), activation='relu'))     # Layer 1
    model.add(Convolution2D(filters=36, kernel_size=(5, 5), strides=(2, 2), activation='relu'))     # Layer 2
    model.add(Convolution2D(filters=48, kernel_size=(5, 5), strides=(2, 2), activation='relu'))     # Layer 3
    model.add(Convolution2D(filters=64, kernel_size=(3, 3), strides=(1, 1), activation='relu'))     # Layer 4
    model.add(Convolution2D(filters=64, kernel_size=(3, 3), strides=(1, 1), activation='relu'))     # Layer 4
    model.add(Flatten())                                                                            # Layer 5
    model.add(Dense(1024), activation='relu')                                                       # Layer 6
    model.add(Dense(128), activation='relu')                                                        # Layer 7
    model.add(Dense(10), activation='relu')                                                         # Layer 8
    model.add(Dense(1))                                                                             # Layer 9
    model.compile(optimizer='adam', loss='mse')
    return model

C:\Users\mschr\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


TypeError: __new__() got an unexpected keyword argument 'serialized_options'

In [ ]:
model = simple_model()    
history_object = model.fit(x_train, y_train, validation_split=0.2, shuffle=True, epochs=50, verbose=1)
model.save('simple_model.h5')
plot_history(history_object, './images/', 'simple_model')

model = lenet_model()    
history_object = model.fit(x_train, y_train, validation_split=0.2, shuffle=True, epochs=50, verbose=1)
model.save('lenet_model.h5')
plot_history(history_object, './images/', 'lenet_model')

model = nvidia_model()    
history_object = model.fit(x_train, y_train, validation_split=0.2, shuffle=True, epochs=50, verbose=1)
model.save('nvidia_model.h5')
plot_history(history_object, './images/', 'nvidia_model')

In [ ]:
# compile and train the model using the generator function
train_samples, validation_samples = train_test_split(samples, test_size=0.2)
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

print(len(train_samples)/32, len(validation_samples)/32)

In [ ]:
model = simple_model()
history_object = model.fit_generator(train_generator, 
                               steps_per_epoch=len(train_samples), 
                               epochs=50, 
                               verbose=1, 
                               callbacks=None, 
                               validation_data=validation_generator, 
                               validation_steps=len(validation_samples), 
                               class_weight=None, 
                               max_queue_size=10, 
                               workers=1, 
                               use_multiprocessing=False, 
                               shuffle=True,
                               initial_epoch=0)

plot_history(history_object, './images/', 'simple_model2')
model.save('simple_model2.h5')

#model = lenet_model()    
#history_object = model.fit_generator(train_generator, samples_per_epoch= len(train_samples), validation_data=validation_generator, nb_val_samples=len(validation_samples), epochs=10)
#plot_history(history_object, './images/', 'lenet_model2'))
#model.save('lenet_model2.h5')

#model = nvidia_model()    
#history_object = model.fit_generator(train_generator, samples_per_epoch= len(train_samples), validation_data=validation_generator, nb_val_samples=len(validation_samples), epochs=10)
#plot_history(history_object, './images/', 'nvidia_model2'))
#model.save('nvidia_mode2l.h5')